In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from PIL import Image
import gc

# Let's predict the adoptability of pets!
Please tell me if I make mistake

## Notes
In this competition you will **predict the speed at which a pet is adopted**, based on the pet’s listing on PetFinder.   
Sometimes a profile represents a group of pets. In this case, the speed of adoption is determined by the speed at which all of the pets are adopted.  
**The data included text, tabular, and image data**. See below for details.   
This is a Kernels-only competition.  
At the end of the competition, test data will be replaced in their entirety with new data of approximately the same size, and **your kernels will be rerun on the new data**.  

## File descriptions
* train.csv - Tabular/text data for the training set
* test.csv - Tabular/text data for the test set
* sample_submission.csv - A sample submission file in the correct format
* breed_labels.csv - Contains Type, and BreedName for each BreedID. Type 1 is dog, 2 is cat.
* color_labels.csv - Contains ColorName for each ColorID
* state_labels.csv - Contains StateName for each StateID

# train/test.csv  
Tabular/text data for the training set  

## Data Fields
* PetID - Unique hash ID of pet profile
* AdoptionSpeed - Categorical speed of adoption. **Lower is faster.** <font color="red">This is the value to predict. </font> See below section for more info.
* Type - Type of animal (1 = Dog, 2 = Cat)
* Name - Name of pet (Empty if not named)
* Age - Age of pet when listed, in months
* Breed1 - Primary breed of pet (Refer to BreedLabels dictionary)
* Breed2 - Secondary breed of pet, if pet is of mixed breed (Refer to BreedLabels dictionary)
* Gender - Gender of pet (1 = Male, 2 = Female, 3 = Mixed, if profile represents group of pets)
* Color1 - Color 1 of pet (Refer to ColorLabels dictionary)
* Color2 - Color 2 of pet (Refer to ColorLabels dictionary)
* Color3 - Color 3 of pet (Refer to ColorLabels dictionary)
* MaturitySize - Size at maturity (1 = Small, 2 = Medium, 3 = Large, 4 = Extra Large, 0 = Not Specified)
* FurLength - Fur length (1 = Short, 2 = Medium, 3 = Long, 0 = Not Specified)
* Vaccinated - Pet has been vaccinated (1 = Yes, 2 = No, 3 = Not Sure)
* Dewormed - Pet has been dewormed (1 = Yes, 2 = No, 3 = Not Sure)
* Sterilized - Pet has been spayed / neutered (1 = Yes, 2 = No, 3 = Not Sure)
* Health - Health Condition (1 = Healthy, 2 = Minor Injury, 3 = Serious Injury, 0 = Not Specified)
* Quantity - Number of pets represented in profile
* Fee - Adoption fee (0 = Free)
* State - State location in Malaysia (Refer to StateLabels dictionary)
* RescuerID - Unique hash ID of rescuer
* VideoAmt - Total uploaded videos for this pet
* PhotoAmt - Total uploaded photos for this pet
* Description - Profile write-up for this pet. The primary language used is English, with some in Malay or Chinese.

In [ ]:
train_df = pd.read_csv("../input/train/train.csv")
test_df = pd.read_csv("../input/test/test.csv")

In [ ]:
print("train.csv shape is {}".format(train_df.shape))
print("test.csv shape is {}".format(test_df.shape))

In [ ]:
train_df.head()

In [ ]:
test_df.head()

## check Null

In [ ]:
train_df.isnull().sum()

In [ ]:
test_df.isnull().sum()

train and test data have **some null in Name and Desctiption columns**  

## AdoptionSpeed (Target)
Contestants are required to predict this value.  
The value is determined by how quickly, if at all, a pet is adopted. 
The values are determined in the following way:   
0 - Pet was adopted on the same day as it was listed.   
1 - Pet was adopted between 1 and 7 days (1st week) after being listed.   
2 - Pet was adopted between 8 and 30 days (1st month) after being listed.   
3 - Pet was adopted between 31 and 90 days (2nd & 3rd month) after being listed.   
4 - No adoption after 100 days of being listed. (There are no pets in this dataset that waited between 90 and 100 days).  

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))
ax.set_title("AdoptionSpeed count in train")
sns.countplot(x="AdoptionSpeed", data=train_df, ax=ax)

## Type
Type of animal (1 = Dog, 2 = Cat) 

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.set_title("Type count in train data")
#ax1.patch.set_facecolor("blue")
#ax1.patch.set_alpha(0.3)
ax2.set_title("Type count in test data")
#ax2.patch.set_facecolor("yellow")
#ax2.patch.set_alpha(0.3)
g = sns.catplot(data=train_df, x="Type", kind="count", ax=ax1)
plt.close(g.fig)
g = sns.catplot(data=test_df, x="Type", kind="count", ax=ax2)
plt.close(g.fig)

In [ ]:
fig, ax1 = plt.subplots(figsize=(8, 5))
#ax1.set_title("train data")
#ax1.patch.set_facecolor("blue")
#ax1.patch.set_alpha(0.3)
g = sns.catplot(data=train_df, x="Type", hue="AdoptionSpeed", kind="count", ax=ax1)
plt.close(g.fig)

The number of Type=1(Dog) is more than Type=2(Cat) in train  
test show a reverse trend  
But this different is not big  

## Name
Name of pet (Empty if not named)

In [ ]:
print("The amount of unique Name is {} in train".format(train_df.Name.nunique()))
print("train shape is {}".format(train_df.shape))
print("The amount of unique Name is {} in test".format(test_df.Name.nunique()))
print("test shape is {}".format(test_df.shape))

In [ ]:
train_df.Name

Some name is duplicated  
And we can see many type of names  
NaN is not Named  
But "No Name Yet" and "No Name" is name like the book "I am a CAT"?  

## Age
Age of pet when listed, in months  

In [ ]:
print("Age in train")
print(train_df.Age.describe())
print("\n\n")
print("Age in test")
print(test_df.Age.describe())

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7), sharex=True) 
sns.distplot(train_df.Age, ax=ax1, bins=20, kde=False)
sns.distplot(test_df.Age, ax=ax2, bins=20, kde=False)
ax1.set_title("Age histgram(bin=10) in train data", fontsize=15)
ax2.set_title("Age histgram(bin=10) in test data", fontsize=15)
ax1.tick_params(labelsize=14)
ax2.tick_params(labelsize=14)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax = sns.stripplot(x="AdoptionSpeed", y="Age", data=train_df, ax=ax)

Most pet is under 100 months old(abount 8)  
but Max is 255 months old(about 21)  
we can show a weak correlation  

## Breed1,2(種類，犬種)
* Breed1 - Primary breed of pet (Refer to BreedLabels dictionary)
* Breed2 - Secondary breed of pet, if pet is of mixed breed (Refer to BreedLabels dictionary)

In [ ]:
train_df.loc[:, ["Breed1", "Breed2"]].head(10)

In [ ]:
train_df.loc[:, ["Breed1", "Breed2"]].describe()

In [ ]:
test_df.loc[:, ["Breed1", "Breed2"]].describe()

In [ ]:
print("The number of mixed in train: {}({:.2%})".format(train_df.query("Breed2 != 0").Type.count(), train_df.query("Breed2 != 0").Type.count()/len(train_df)))
print("The number of mixed in test: {}({:.2%})".format(test_df.query("Breed2 != 0").Type.count(), test_df.query("Breed2 != 0").Type.count()/len(test_df)))

### Let's check breed label
* breed_labels.csv - Contains Type, and BreedName for each BreedID. Type 1 is dog, 2 is cat.

In [ ]:
breed_df = pd.read_csv("../input/breed_labels.csv")
breed_df.head()

In [ ]:
breed_df.tail()

In [ ]:
fig, ax = plt.subplots()
sns.countplot(x="Type", data=breed_df, ax=ax)
g = ax.set_title("Breeder type count")

BreedID starts from 1  to 306  
but some pet's Breed1 is 0  
this means the pet primary breed is unknow?  
and some pet's Breed1 is 307  
BreedID:307 is not in breed label  
so BreedID = 0 or 307 means unknown?  

In [ ]:
print("The number of Breed1=0 in train: {}".format(train_df.query("Breed1 == 0").Type.count()))
print("The number of Breed1=0 in test: {}".format(test_df.query("Breed1 == 0").Type.count()))
print("The number of Breed1=307 in train: {}".format(train_df.query("Breed1 == 307").Type.count()))
print("The number of Breed1=307 in test: {}".format(test_df.query("Breed1 == 307").Type.count()))

In [ ]:
train_df.query("Breed1 == 0")

In [ ]:
train_df.query("Breed1 == 307").head()

## Gender 
Gender of pet (1 = Male, 2 = Female, 3 = Mixed, if profile represents group of pets)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
sns.countplot(x="Gender", data=train_df, ax=ax1)
sns.countplot(x="Gender", data=test_df, ax=ax2)
ax1.set_title("The amount of each Gender in train data", fontsize=15)
g = ax2.set_title("The amount of each Gender in test data", fontsize=15)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax = sns.countplot(x="Gender", data=train_df, ax=ax, hue="AdoptionSpeed")

there are same distribution in train/test

## Color1 ,2 ,3
* Color1 -  Color 1 of pet (Refer to ColorLabels dictionary)
* Color2 - Color 2 of pet (Refer to ColorLabels dictionary)
* Color3 - Color 3 of pet (Refer to ColorLabels dictionary)

In [ ]:
train_df.loc[:, ["Color1", "Color2", "Color3"]].head(10)

In [ ]:
train_df.loc[:, ["Color1", "Color2", "Color3"]].describe()

In [ ]:
test_df.loc[:, ["Color1", "Color2", "Color3"]].describe()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
ax1 = sns.countplot(x="Color1", data=train_df, hue="AdoptionSpeed", ax=ax1)
ax2 = sns.countplot(x="Color1", data=test_df, ax=ax2)

### Let's check color labels
* Contains ColorName for each ColorID

In [ ]:
color_df = pd.read_csv("../input/color_labels.csv")

In [ ]:
color_df

In [ ]:
print("The number of nothing color(color1,2,3 == 0) pet: {}".format(train_df.query("Color1==0 & Color2==0 & Color3==0").shape[0]))

## MaturitySize 
Size at maturity (1 = Small, 2 = Medium, 3 = Large, 4 = Extra Large, 0 = Not Specified)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1 = sns.countplot(x="MaturitySize", data=train_df, hue="AdoptionSpeed", ax=ax1)
ax2 = sns.countplot(x="MaturitySize", data=test_df, ax=ax2)
ax1.set_title("The amount of each MaturitySize in train data", fontsize=15)
g = ax2.set_title("The amount of each MaturitySize in test data", fontsize=15)

MaturitySize == 0(Not Specified) is not exist  

## FurLength
Fur length (1 = Short, 2 = Medium, 3 = Long, 0 = Not Specified)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1 = sns.countplot(x="FurLength", data=train_df, hue="AdoptionSpeed", ax=ax1)
ax2 = sns.countplot(x="FurLength", data=test_df, ax=ax2)
ax1.set_title("The amount of each FurLength in train data", fontsize=15)
g = ax2.set_title("The amount of each FurLength in test data", fontsize=15)

FurLength == 0(Not Specified) is not exist  

## Vaccinated(予防接種)
Pet has been vaccinated (1 = Yes, 2 = No, 3 = Not Sure)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1 = sns.countplot(x="Vaccinated", data=train_df, hue="AdoptionSpeed", ax=ax1)
ax2 = sns.countplot(x="Vaccinated", data=test_df, ax=ax2)
ax1.set_title("The amount of each Vaccinated in train data", fontsize=15)
g = ax2.set_title("The amount of each Vaccinated in test data", fontsize=15)

## Dewormed(虫下し)
Pet has been dewormed (1 = Yes, 2 = No, 3 = Not Sure)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1 = sns.countplot(x="Dewormed", data=train_df, hue="AdoptionSpeed", ax=ax1)
ax2 = sns.countplot(x="Dewormed", data=test_df, ax=ax2)
ax1.set_title("The amount of each Dewormed in train data", fontsize=15)
g = ax2.set_title("The amount of each Dewormed in test data", fontsize=15)

## Sterilized(不妊手術)
Pet has been spayed / neutered (1 = Yes, 2 = No, 3 = Not Sure)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1 = sns.countplot(x="Sterilized", data=train_df, hue="AdoptionSpeed", ax=ax1)
ax2 = sns.countplot(x="Sterilized", data=test_df, ax=ax2)
ax1.set_title("The amount of each Sterilized in train data", fontsize=15)
g = ax2.set_title("The amount of each Sterilized in test data", fontsize=15)

Late adoption speed pet is high ratio in sterilized  
Adoption speed ==2 pet is high ration in not sterilized  

## Health
Health Condition (1 = Healthy, 2 = Minor Injury, 3 = Serious Injury, 0 = Not Specified)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1 = sns.countplot(x="Health", data=train_df, hue="AdoptionSpeed", ax=ax1)
ax2 = sns.countplot(x="Health", data=test_df, ax=ax2)
ax1.set_title("The amount of each Health in train data", fontsize=15)
g = ax2.set_title("The amount of each Health in test data", fontsize=15)

Most pet is good Health  
Not Healty pet is not fast adoption speed

## Quantity
Number of pets represented in profile

In [ ]:
train_df.Quantity.describe()

In [ ]:
test_df.Quantity.describe()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1 = sns.distplot(train_df.Quantity, kde=False, bins=20, ax=ax1)
ax2 = sns.distplot(test_df.Quantity, kde=False, bins=20, ax=ax2)
ax1.set_title("The amount of Quantity in train data", fontsize=15)
g = ax2.set_title("The amount of Quantity in test data", fontsize=15)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax = sns.stripplot(x="Quantity", y="AdoptionSpeed", data=train_df, ax=ax)

Many quantity is not fast AdoptionSpeed

## Fee
Adoption fee (0 = Free)

In [ ]:
train_df.Fee.describe()

In [ ]:
test_df.Fee.describe()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1 = sns.distplot(train_df.Fee, kde=False, bins=20, ax=ax1)
ax2 = sns.distplot(test_df.Fee, kde=False, bins=20, ax=ax2)
ax1.set_title("The amount of Fee in train data", fontsize=15)
g = ax2.set_title("The amount of Fee in test data", fontsize=15)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax = sns.stripplot(x="Fee", y="AdoptionSpeed", data=train_df, ax=ax)
#ax.set_xticks(np.arange(0, 10))

## State
State location in Malaysia (Refer to StateLabels dictionary)

In [ ]:
train_df.State.head(10)

In [ ]:
test_df.State.head(10)

### Let's check StateLabels
* state_labels.csv - Contains StateName for each StateID

In [ ]:
state_df = pd.read_csv("../input/state_labels.csv")
state_df.head()

In [ ]:
print("StateID min in train: {}".format(train_df.State.min()))
print("StateID max in train: {}".format(train_df.State.max()))
print("StateID min in test: {}".format(test_df.State.min()))
print("StateID max in test: {}".format(test_df.State.max()))
print("StateID min in state_labels: {}".format(state_df.StateID.min()))
print("StateID max in state_labels: {}".format(state_df.StateID.max()))

## RescuerID
Unique hash ID of rescuer

In [ ]:
print("train data num is {}, the number of unique RescureID is {}".format(train_df.shape[0], train_df.RescuerID.nunique()))
print("test data num is {}, the number of unique RescureID is {}".format(test_df.shape[0], test_df.RescuerID.nunique()))

In [ ]:
sorted(train_df.groupby(["RescuerID"]).Type.count().values, reverse=True)[:15]

some rescuer rescued too many times

## VideoAmt
Total uploaded videos for this pet

In [ ]:
train_df.VideoAmt.describe()

In [ ]:
test_df.VideoAmt.describe()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), sharex=True)
ax1 = sns.distplot(train_df.VideoAmt, kde=False, bins=9, ax=ax1)
ax2 = sns.distplot(test_df.VideoAmt, kde=False, bins=9, ax=ax2)
ax1.set_title("The amount of VideoAmt in train data", fontsize=15)
g = ax2.set_title("The amount of VideoAmt in test data", fontsize=15)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax = sns.stripplot(x="VideoAmt", y="AdoptionSpeed", data=train_df, ax=ax)
#ax.set_xticks(np.arange(0, 10))

## PhotoAmt
Total uploaded photos for this pet

In [ ]:
train_df.PhotoAmt.describe()

In [ ]:
test_df.PhotoAmt.describe()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5), sharex=True)
ax1 = sns.distplot(train_df.PhotoAmt, kde=False, bins=30, ax=ax1)
ax2 = sns.distplot(test_df.PhotoAmt, kde=False, bins=30, ax=ax2)
ax1.set_title("The amount of PhotoAmt in train data", fontsize=15)
g = ax2.set_title("The amount of PhotoAmt in test data", fontsize=15)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
ax = sns.stripplot(x="PhotoAmt", y="AdoptionSpeed", data=train_df, ax=ax)
#ax.set_xticks(np.arange(0, 10))

## Description
Profile write-up for this pet. The primary language used is English, with some in Malay or Chinese.

In [ ]:
train_df.Description.head()

In [ ]:
test_df.Description.head()

## PairPlot

In [ ]:
g = pd.plotting.scatter_matrix(train_df, alpha=0.8, figsize=(12,12), range_padding=0.5)

## correlation headmap

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9)) 
ax = sns.heatmap(train_df.corr(), square=True, vmax=1, vmin=-1, center=0)

# Images

In [ ]:
fig, axes = plt.subplots(4,3, figsize=(15, 20))
images_train = os.listdir("../input/train_images/")
fig.suptitle("train images")
for i, img in enumerate(np.random.choice(images_train, 12)):
    image = Image.open("../input/train_images/" + img)
    axes[i//3, i%3].imshow(image)

In [ ]:
fig, axes = plt.subplots(4,3, figsize=(15, 20))
images_train = os.listdir("../input/test_images/")
fig.suptitle("test images")
for i, img in enumerate(np.random.choice(images_train, 12)):
    image = Image.open("../input/test_images/" + img)
    axes[i//3, i%3].imshow(image)

# So Cute!!!!!

Thank you for watching my kernel.  
I hope you will find the kernel useful.  
# In Progress...
* add contemt  
* make baseline model
* fix figure axis error